<a href="https://colab.research.google.com/github/leoscarlato/math-language-project/blob/main/Projeto_Linguagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOME da linguagem: **{mat}**

POR: Alexandre Magno e Leonardo Scarlato


**{mat}** consiste em uma linguagem de programação interpretada, com a finalidade de facilitar a escrita em português de operações matemáticas. Nesta linguagem, há palavras reservadas que representam operações que proximam a dialética matemática oral com o código. Assim, proporciando um espaço mais amigavél para estudos de ambas as ciências.

O público alvo dessa linguagem são de jovens no estágio de desenvolvimento do estudo matemático e de programação.

**{mat}** lidará com números pertencentes ao conjunto dos Racionais(Q). Contudo, na linguagem, trabalha-se com a separação entre inteiros (int) e fracionarios (float).

In [3]:
import numpy as np
import math

**Analisador Lexico**

In [4]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add("NUMBER", r"\d+(.\d)?")
lg.add("PLUS", r"\+")
lg.add("MINUS", r"\-")
lg.add("MUL", r"\*")
lg.add("DIV", r"/")
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')
lg.add('OPEN_BRACE', r'\{')
lg.add('CLOSE_BRACE', r'\}')

lg.add("IF", r'if')
lg.add("ELSE", r'else')
lg.add("PRINT", r'print')
lg.add("ROOT", r"raiz_de")
lg.add("POW", r"elevado_a")
lg.add("FACT", r"fatorial")
lg.add("PI", r"pi")
lg.add("SIN", r"seno")
lg.add("COS", r"cosseno")
lg.add("TAN", r"tangente")
lg.add("WHILE", r"while")
lg.add("FOR", r"for")
lg.add("SEMICOL", r"\;")

lg.add("RETURN", r'\:')
lg.add("FUNC", r"funcao")
lg.add("TYPE", r"inteiro")
lg.add("TYPE", r"fracionario")

lg.add('ID', r'[a-zA-z][a-zA-z0-9]*')
lg.add('COMP',r'==')
lg.add('COMP',r'!=')
lg.add('COMP',r'>=')
lg.add('COMP',r'>')
lg.add('COMP',r'<=')
lg.add('COMP',r'<')
lg.add('EQUALS', r"=")

lg.ignore('\s+')

lexer = lg.build()

**Classes da Arvore Sintatica**

In [5]:
from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self,decls,stmts):
        self.decls = decls
        self.stmts = stmts
    
    def accept(self, visitor):
        visitor.visit_prog(self)

class Declatations(BaseBox):
    def __init__(self,decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_declarations(self)

class Declaration(BaseBox):
    def __init__(self,typee,id,expr=None):
        self.id = id
        self.typee = typee
        self.expr = expr
        
    def accept(self, visitor):
        visitor.visit_declaration(self)

class Statements(BaseBox):
    def __init__(self,stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self,id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)

class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1

    def accept(self, visitor):
        visitor.visit_while(self)

class For(BaseBox):
    def __init__(self, atrib1, expr1, comp, expr2, atrib2, ie1):
        self.atrib1=atrib1
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.atrib2=atrib2
        self.ie1=ie1

    def accept(self, visitor):
        visitor.visit_for(self)

class Function(BaseBox):
    def __init__(self, id, param, expr):
        self.id = id
        self.param = param
        self.expr = expr
    
    def accept(self, visitor):
        visitor.visit_function(self)

class CallFunction(BaseBox):
    def __init__(self, id, param):
        self.id = id
        self.param = param
    
    def accept(self, visitor):
        return visitor.visit_callfunction(self)

class Print(BaseBox):
    def __init__(self, expr):
        self.expr=expr
    
    def accept(self, visitor):
        visitor.visit_print(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        return visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value

class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    pass

class Sub(BinaryOp):
    pass

class Mul(BinaryOp):
    pass

class Div(BinaryOp):
    pass

class Pow(BinaryOp):
    pass

class Root(BinaryOp):
    pass

class Fact(BinaryOp):
    pass

class Sin(BinaryOp):
    pass

class Cos(BinaryOp):
    pass

class Tan(BinaryOp):
    pass

**Analisador Sintático**

In [6]:
from rply import ParserGenerator

pg = ParserGenerator(
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'ID', 'WHILE', 'TYPE',
     'EQUALS','COMP','IF','ELSE', 'WHILE', 'ROOT', 'FOR', 'FUNC', 'RETURN',
     'POW', 'FACT', 'PI', 'SEMICOL', 'PRINT', 'OPEN_BRACE', 'CLOSE_BRACE'
    ],

    precedence = [
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV']),
        ('left', ['ROOT', 'POW']),
    ]
)

@pg.production('prog : declarations statements')
def prog(p):
    return Prog(p[0], p[1])

@pg.production('prog : declarations')
def prog(p):
    return Prog(p[0], None)

# --- ---

@pg.production('declarations : declaration')
def declarations(p):
    return Declatations(p[0], None)

@pg.production('declarations : declaration declarations')
def declarations(p):
    return Declatations(p[0], p[1])

@pg.production('declaration : TYPE ID SEMICOL')
def declaration(p):
    return Declaration(p[0].getstr(), p[1].getstr())

@pg.production('declaration : TYPE ID EQUALS expression SEMICOL')
def declaration(p):
    return Declaration(p[0].getstr(), p[1].getstr(), p[3])

@pg.production('declaration : FUNC ID OPEN_PARENS ID CLOSE_PARENS RETURN expression SEMICOL')
def declaration_function(p):
    return Function(p[1].getstr(), f"__{p[3].getstr()}__", p[6])

# --- statements --- #

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : PRINT OPEN_PARENS expression CLOSE_PARENS SEMICOL')
def expression_id(p):
    return Print(p[2])


@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_BRACE statements CLOSE_BRACE ELSE OPEN_BRACE statements CLOSE_BRACE')
def expression_ifelse1(p):
    return IfElse(p[2],p[3].getstr(),p[4],p[7],p[11])

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_BRACE statements CLOSE_BRACE')
def expression_ifelse1(p):
    return IfElse(p[2],p[3].getstr(),p[4],p[7],None)

@pg.production('openstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_BRACE statements CLOSE_BRACE')
def statement_while(p):
    return While(p[2],p[3].getstr(),p[4],p[7])

@pg.production('openstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def statement_while(p):
    return While(p[2],p[3].getstr(),p[4],p[6])

@pg.production('openstatement : FOR OPEN_PARENS ID EQUALS expression SEMICOL expression COMP expression SEMICOL ID EQUALS expression CLOSE_PARENS OPEN_BRACE statements CLOSE_BRACE')
def statement_for(p):
    return For(Atrib(p[2].getstr(),p[4]),p[6],p[7].getstr(),p[8],Atrib(p[10].getstr(),p[12]),p[15])

@pg.production('openstatement : FOR OPEN_PARENS ID EQUALS expression SEMICOL expression COMP expression SEMICOL ID EQUALS expression CLOSE_PARENS openstatement')
def statement_for(p):
    return For(Atrib(p[2].getstr(),p[4]),p[6],p[7].getstr(),p[8],Atrib(p[10].getstr(),p[12]),p[14])


@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(eval(p[0].getstr()))

@pg.production('expression : MINUS NUMBER')
def expression_number(p):
    return Number(-eval(p[1].getstr()))

@pg.production('expression : PI')
def expression_number(p):
    return Number(np.pi)

@pg.production('expression : ID OPEN_PARENS expression CLOSE_PARENS')
def expression_number(p):
    return CallFunction(p[0].getstr(), p[2])

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
@pg.production('expression : expression ROOT expression')
@pg.production('expression : expression POW expression')
@pg.production('expression : expression FACT')
def expression_binop(p):
    left = p[0]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, p[2])
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, p[2])
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, p[2])
    elif p[1].gettokentype() == 'DIV':
        return Div(left, p[2])
    elif p[1].gettokentype() == 'ROOT':
        return Root(left, p[2])
    elif p[1].gettokentype() == 'POW':
        return Pow(left, p[2])
    elif p[1].gettokentype() == 'FACT':
        return Fact(left, None)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [7]:
code = """
funcao raiz_cubica(num): num raiz_de 3; 
funcao raiz_quadrada(num): num raiz_de 2; 
print(raiz_quadrada(16));
print(raiz_cubica(27));
"""
arvore = parser.parse(lexer.lex(code))

In [8]:
class Visitor(BaseBox):
    pass

In [9]:
ST = {}
VAR = {}
FNC = {}

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_declarations(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_declaration(self, d):
        ST[d.id]=d.typee
        VAR[d.id]=0

    def visit_function(self, f):
        ST[f.id] = "func"
        VAR[f.param] = 0
        FNC[f.id] = f

In [10]:
arvore.accept(SymbolTable())
print(ST)

{'raiz_cubica': 'func', 'raiz_quadrada': 'func'}


In [11]:
class Decorator(Visitor):
    def visit_prog(self, p):
        p.decls.accept(self)
        if p.stmts:
            p.stmts.accept(self)
    
    def visit_declarations(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)
    
    def visit_declaration(self, d):
        if d.expr:
            d.decor_type=ST[d.id]
            d.expr.accept(self)
    
    def visit_function(self, f):
        pass

    def visit_callfunction(self, f):
        f.decor_type = "fracionario"
        f.param.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
            d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)
    
    def visit_atrib(self, i):
        if i.id in ST:
            i.decor_type=ST[i.id]
        else:
            raise AssertionError(f'{i.id} not declared')
        i.expr.accept(self)
    
    def visit_print(self, i):
        pass
    
    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
            i.ie2.accept(self)
    
    def visit_for(self, i):
        i.atrib1.accept(self)
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.atrib2.accept(self)
        i.ie1.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
    
    def visit_id(self, i):
        if i.value in ST:
            i.decor_type=ST[i.value]
        else:
            raise AssertionError(f'id: {i.value} not declared')

    def visit_number(self, i):
        i.decor_type= "inteiro" if i.value.__class__.__name__ == "int" else "fracionario"
    
    def binaryOpVisit(self, a):
        a.left.accept(self)
        a.right.accept(self)
        a.decor_type = a.left.decor_type

    def visit_add(self, a):
        self.binaryOpVisit(a)
    
    def visit_sub(self, a):
        self.binaryOpVisit(a)

    def visit_mul(self, a):
        self.binaryOpVisit(a)

    def visit_div(self, a):
        self.binaryOpVisit(a)
    
    def visit_root(self, a):
        self.binaryOpVisit(a)

    def visit_pow(self, a):
        self.binaryOpVisit(a)

    def visit_fact(self, a):
        a.left.accept(self)
        a.decor_type = a.left.decor_type

In [12]:
arvore.accept(Decorator())

In [13]:
class TypeVerifier(Visitor):
    def visit_prog(self, p):
        p.decls.accept(self)
        if p.stmts:
            p.stmts.accept(self)
    
    def visit_declarations(self, d):
        d.decl.accept(self)
        if d.decls!=None:
            d.decls.accept(self)
    
    def visit_declaration(self, d):
        pass

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
            d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)
    
    def visit_function(self, f):
        pass

    def visit_callfunction(self, f):
        pass
    
    def visit_print(self, i):
        pass

    def visit_atrib(self, i):
        pass

    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
            raise AssertionError('type error')

    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
            raise AssertionError(f'type error {i.expr1.decor_type} & {i.expr2.decor_type}')
    
    def visit_for(self, i):
        if i.atrib1.decor_type!=i.atrib1.expr.decor_type:
          raise AssertionError('type error')

        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

        if i.atrib1.decor_type!=i.atrib1.expr.decor_type:
          raise AssertionError('type error')

In [14]:
arvore.accept(TypeVerifier())

In [15]:
class Eval(Visitor):
    def visit_prog(self, p):
        p.decls.accept(self)
        if p.stmts:
            p.stmts.accept(self)
    
    def visit_declarations(self, d):
        d.decl.accept(self)
        if d.decls!=None:
            d.decls.accept(self)
    
    def visit_declaration(self, d):
        if d.expr:
            num = d.expr.accept(self)
            VAR[d.id] = int(num) if ST[d.id] == "inteiro" else float(num)

    def visit_function(self, f):
        pass

    def visit_callfunction(self, c):
        function = FNC.get(c.id)
        if function:
            VAR[function.param] = c.param.accept(self)
            value = function.expr.accept(self)
            del VAR[function.param]
            return value
        raise AssertionError(f'{c.id} not declared')

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
            d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)
    
    def visit_while(self, d):
        while(eval(f"{d.expr1.accept(self)} {d.comp} {d.expr2.accept(self)}")):
            d.ie1.accept(self)

    def visit_for(self, d):
        d.atrib1.accept(self)
        while(eval(f"{d.expr1.accept(self)} {d.comp} {d.expr2.accept(self)}")):
            d.ie1.accept(self)
            d.atrib2.accept(self)
    
    def visit_ifelse(self, d):
        if(eval(f"{d.expr1.accept(self)} {d.comp} {d.expr2.accept(self)}")):
            d.ie1.accept(self)
        elif d.ie2!=None:
            d.ie2.accept(self)
    
    def visit_print(self, d):
        print(d.expr.accept(self))

    def visit_atrib(self, d):
        num = d.expr.accept(self)
        VAR[d.id] = int(num) if ST[d.id] == "inteiro" else float(num)

    def visit_id(self, d):
        if f'__{d.value}__' in VAR:
            return VAR[f'__{d.value}__']
        elif d.value in VAR:
            return VAR[d.value]
        raise AssertionError(f'{d.value} not declared')

    def visit_number(self, number):
        return number.value

    def visit_add(self, d):
        return d.left.accept(self) + d.right.accept(self)
    
    def visit_sub(self, d):
        return d.left.accept(self) - d.right.accept(self)
    
    def visit_mul(self, d):
        return d.left.accept(self) * d.right.accept(self)

    def visit_div(self, d):
        return d.left.accept(self) / d.right.accept(self)
    
    def visit_root(self, d):
        return d.left.accept(self) ** (1/d.right.accept(self))
    
    def visit_pow(self, d):
        return d.left.accept(self) ** d.right.accept(self)
    
    def visit_fact(self, d):
        return math.factorial(d.left.accept(self))

In [16]:
arvore.accept(Eval())

4.0
3.0


In [17]:
class Show(Visitor):
    def visit_prog(self, p):
        p.decls.accept(self)
        if p.stmts:
            p.stmts.accept(self)
    
    def visit_declarations(self, d):
        d.decl.accept(self)
        if d.decls!=None:
            d.decls.accept(self)
    
    def visit_declaration(self, d):
        if d.expr:
            print(f'{d.decor_type} {d.id}: {d.expr.__class__.__name__}')

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
            d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)
    
    def visit_atrib(self, d):
        print(f'{d.decor_type} {d.id}: {d.expr.__class__.__name__} ')

    def visit_function(self, f):
        print(f'{f.id}({f.param}): {f.expr.__class__.__name__}')

    def visit_while(self, d):
        pass

    def visit_for(self, d):
        pass

    def visit_print(self, i):
        pass


In [18]:
arvore.accept(Show())

raiz_cubica(__num__): Root
raiz_quadrada(__num__): Root


In [19]:
for k in VAR.keys():
    print(f'{ST[k]} {k} = {VAR[k]}')


In [20]:
code2 = "inteiro a = 12 elevado_a 2; print(a);"
code2 = "inteiro a = 0; while(a < 100) a = a + 10; print(a);"
code2 = "inteiro a; for(a = 0; a < 10; a = a + 1) print(a);"
code2 = "inteiro a = 0; inteiro b = 1; for(a = 5; a > 0; a = a - 1) b = b*a; print(b);"

In [21]:
code2 = """

inteiro a = 0; 
fracionario soma = 0; 
fracionario value = 1;

for(a = 10; a > 0; a = a-1){
    if(a < 5){
        if(a == 0){
            print(0);
        }
        value = a fatorial;
    } else {
        value = a raiz_de 2;
    }
    print(value);
    soma = soma + value;
}

"""

In [22]:
code2 = """
inteiro num = -10;
funcao raiz_quadrada(num): num raiz_de 2; 
fracionario p = raiz_quadrada(16);
print(p);
print(num);

"""

In [23]:
arvore2 = parser.parse(lexer.lex(code2))
arvore2.accept(SymbolTable())
arvore2.accept(Decorator())
arvore2.accept(TypeVerifier())
arvore2.accept(Eval())

4.0
-10
